In [44]:
### Notebook for scraping college football data from sports-reference.com

# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

import time
import random

import tqdm as tqdm


# URL of page to be scraped
BASE_URL ='https://www.sports-reference.com/cfb/years/'
END_URL = '-schedule.html'

year_list = list(range(2013, 2015))

min_dealy = 3 # site terms stay they kick if request >20 per minute

# https://www.sports-reference.com/cfb/years/1988-schedule.html

In [45]:
## Block for scraping Wikipedia for List of d1 college football teams

# URL of page to be scraped
# https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_FBS_football_programs

# Extract table from wikipedia page
wiki_url = 'https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_FBS_football_programs'
wiki_table = pd.read_html(wiki_url)[0]

wiki_table.head()

# ## save list of teams to csv
# wiki_table.to_csv('TEMP/cfb_d1_teams.csv', index=False)

,School,Nickname,City,State [2],Enrollment,CurrentConference,FormerConferences,FirstYear,JoinedFBS
0,Air Force,Falcons,USAF AcademyColorado Springs[n 1],CO,4181,Mountain West,WAC,1955,1957
1,Akron,Zips,Akron,OH,14516,MAC,"OAC, MCC, OVC",1891,1987
2,Alabama,Crimson Tide,Tuscaloosa,AL,38316,SEC,"SIAA, SoCon",1892,NaN
3,Appalachian State,Mountaineers,Boone,NC,20641,Sun Belt,"NSC, SoCon",1928,2014[n 2]
4,Arizona,Wildcats,Tucson,AZ,49471,Pac-12[n 3],"BIAA, WAC",1899,NaN


In [46]:
import time
def scrape_year_data(year):
    """Scrape game data for a given year from sports-reference.com."""
    # Construct URL for the given year
    url = BASE_URL + str(year) + END_URL
    
    # Try reading the table into a pandas DataFrame
    try:
        df = pd.read_html(url)
        df = pd.DataFrame(df[0])  # Convert response from list to dataframe
    except:
        return None  # Return None if scraping fails
    
    # Clean the dataframe of rows that are not games
    df = df[df['Wk'] != 'Wk']
    
    # Add a 'Year' column to the dataframe to keep track of the year
    df['Season'] = year
    
    return df

# This is the finalized scraping function.


def scrape_all_years(start_year, end_year):
    """Scrape game data for all years in the given range."""
    
    # Initialize an empty master dataframe
    master_df = pd.DataFrame()
    
    for year in range(start_year, end_year + 1):
        # Scrape data for the current year
        year_data = scrape_year_data(year)
        
        if year_data is not None:
            # Append the data to the master dataframe
            master_df = master_df.append(year_data, ignore_index=True)
        
        # Save to CSV every 10 years as a backup
        if year % 10 == 0:
            master_df.to_csv(f"../TEMP/data_backup_{year}.csv", index=False)
        
        # Sleep for 3 seconds to respect rate limits
        time.sleep(3.5)
    
    # Save the entire data at the end
    master_df.to_csv("../data/SCRAPE_TEST1.csv", index=False)
    
    return master_df

# The function is ready to be used. You can call it with the desired year range when running it on your machine.


In [47]:
# data = scrape_all_years(1869, 2022)

In [48]:
## Load data from completed scrape
data = pd.read_csv('../data/cfb_scrape_raw.csv')

C:\Users\Justin\AppData\Local\Temp\ipykernel_7268\2456455489.py:2: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/cfb_scrape_raw.csv')


In [49]:
### Extract the ranking from the team name where applicable
## Pool ranking is contained in parentheses at the beginning of the team name string of the Winner and Loser columns
def extract_ranking(team_name):
    """Extract the ranking from the team name where applicable."""
    # If the team name does not start with a parenthesis, there is no ranking
    if team_name[0] != '(':
        return None
    
    # Find the closing parenthesis
    closing_paren = team_name.find(')')
    
    # Extract the ranking
    ranking = team_name[1:closing_paren]
    
    return ranking

# Apply function to Winner column and save in new 'Winner_Rank' column
data['Winner_Rank'] = data['Winner'].apply(extract_ranking)
# Apply function to Loser column and save in new 'Loser_Rank' column
data['Loser_Rank'] = data['Loser'].apply(extract_ranking)

# Remove the ranking from the team name
data['Winner'] = data['Winner'].str.replace(r"\(.*\)","")
data['Loser'] = data['Loser'].str.replace(r"\(.*\)","")

# Strip leading and trailing whitespace from team names
data['Winner'] = data['Winner'].str.strip()
data['Loser'] = data['Loser'].str.strip()

# rename columns for clarity
replace_dict = {'RK':'season_index', 'Pts':'Winner_Pts', 'Pts.1':'Loser_Pts', 'Unnamed: 6':'loc_ind', 'Unnamed: 7':'loc_ind2'}

# rename columns
data.rename(columns=replace_dict, inplace=True)

# name column names lower case for consistency
data.columns = data.columns.str.lower()

# Combine the two columns that are location indicators into a single column
# The columns don't overlapp at all so they can be combined into one column
data['loc_ind'] = data['loc_ind'].fillna(data['loc_ind2'])
# drop the loc_ind2 column
data.drop(columns='loc_ind2', inplace=True)

data.info()


C:\Users\Justin\AppData\Local\Temp\ipykernel_7268\2437125262.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Winner'] = data['Winner'].str.replace(r"\(.*\)","")
C:\Users\Justin\AppData\Local\Temp\ipykernel_7268\2437125262.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Loser'] = data['Loser'].str.replace(r"\(.*\)","")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81606 entries, 0 to 81605
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rk           81606 non-null  int64  
 1   wk           81606 non-null  int64  
 2   date         81606 non-null  object 
 3   day          81606 non-null  object 
 4   winner       81606 non-null  object 
 5   winner_pts   81604 non-null  float64
 6   loc_ind      33643 non-null  object 
 7   loser        81606 non-null  object 
 8   loser_pts    81604 non-null  float64
 9   notes        5177 non-null   object 
 10  season       81606 non-null  int64  
 11  time         7354 non-null   object 
 12  winner_rank  14874 non-null  object 
 13  loser_rank   5892 non-null   object 
dtypes: float64(2), int64(3), object(9)
memory usage: 8.7+ MB


In [50]:
# save temp csv file to check data
data.to_csv('../TEMP/cfb_scrape_clean.csv', index=False)
data.columns

Index(['rk', 'wk', 'date', 'day', 'winner', 'winner_pts', 'loc_ind', 'loser',
       'loser_pts', 'notes', 'season', 'time', 'winner_rank', 'loser_rank'],
      dtype='object')

In [51]:
#### Data tranformation to classify home away and neutral site games and segment them out / pivot them 
df = data.copy()
# Initialize new columns
df['HostTeamName'] = None
df['HostScore'] = None
df['HostRank'] = None
df['AwayTeamName'] = None
df['AwayScore'] = None
df['AwayRank'] = None
df['NeutralTeam1'] = None
df['NeutralScore1'] = None
df['NeutralRank1'] = None
df['NeutralTeam2'] = None
df['NeutralScore2'] = None
df['NeutralRank2'] = None

# Determine host and away teams based on loc_ind and notes columns
for index, row in df.iterrows():
    if row['loc_ind'] == '@':
        df.at[index, 'HostTeamName'] = row['loser']
        df.at[index, 'HostScore'] = row['loser_pts']
        df.at[index, 'HostRank'] = row['loser_rank']
        df.at[index, 'AwayTeamName'] = row['winner']
        df.at[index, 'AwayScore'] = row['winner_pts']
        df.at[index, 'AwayRank'] = row['winner_rank']
    elif pd.isna(row['loc_ind']) or row['loc_ind'] == 'N':
        if row['loc_ind'] == 'N':
            df.at[index, 'NeutralTeam1'] = row['winner']
            df.at[index, 'NeutralScore1'] = row['winner_pts']
            df.at[index, 'NeutralRank1'] = row['winner_rank']
            df.at[index, 'NeutralTeam2'] = row['loser']
            df.at[index, 'NeutralScore2'] = row['loser_pts']
            df.at[index, 'NeutralRank2'] = row['loser_rank']
        else:
            df.at[index, 'HostTeamName'] = row['winner']
            df.at[index, 'HostScore'] = row['winner_pts']
            df.at[index, 'HostRank'] = row['winner_rank']
            df.at[index, 'AwayTeamName'] = row['loser']
            df.at[index, 'AwayScore'] = row['loser_pts']
            df.at[index, 'AwayRank'] = row['loser_rank']




In [52]:
# Assign Results (Win, Lose, Tie) to each game

# Function to determine result based on score comparison
def determine_result(score1, score2):
    if score1 > score2:
        return 'W'
    elif score1 < score2:
        return 'L'
    else:
        return 'T'

# Calculate results for host, away, and neutral teams
df['HostTeamResult'] = df.apply(lambda row: determine_result(row['HostScore'], row['AwayScore']) if pd.notna(row['HostScore']) and pd.notna(row['AwayScore']) else None, axis=1)
df['AwayTeamResult'] = df.apply(lambda row: determine_result(row['AwayScore'], row['HostScore']) if pd.notna(row['HostScore']) and pd.notna(row['AwayScore']) else None, axis=1)
df['NeutralTeam1Result'] = df.apply(lambda row: determine_result(row['NeutralScore1'], row['NeutralScore2']) if pd.notna(row['NeutralScore1']) and pd.notna(row['NeutralScore2']) else None, axis=1)
df['NeutralTeam2Result'] = df.apply(lambda row: determine_result(row['NeutralScore2'], row['NeutralScore1']) if pd.notna(row['NeutralScore1']) and pd.notna(row['NeutralScore2']) else None, axis=1)

# # Display the transformed dataframe with results and notes column
# df[['date', 'HostTeamName', 'HostScore', 'HostRank', 'HostTeamResult', 'AwayTeamName', 'AwayScore', 'AwayRank', 'AwayTeamResult',
#     'NeutralTeam1', 'NeutralScore1', 'NeutralRank1', 'NeutralTeam1Result', 'NeutralTeam2', 'NeutralScore2', 'NeutralRank2', 'NeutralTeam2Result', 'notes']].head()


In [53]:

# Show the transformed dataframe
df[['season','date', 'HostTeamName', 'HostTeamResult', 'HostScore', 'HostRank', 'AwayTeamName', 'AwayTeamResult', 'AwayScore', 'AwayRank',
    'NeutralTeam1', 'NeutralTeam1Result', 'NeutralScore1', 'NeutralRank1', 'NeutralTeam2', 'NeutralTeam2Result', 'NeutralScore2', 'NeutralRank2','notes']].sample(20)

,season,date,HostTeamName,HostTeamResult,HostScore,HostRank,AwayTeamName,AwayTeamResult,AwayScore,AwayRank,NeutralTeam1,NeutralTeam1Result,NeutralScore1,NeutralRank1,NeutralTeam2,NeutralTeam2Result,NeutralScore2,NeutralRank2,notes
70913,2010,"Sep 25, 2010",Michigan,W,65.0,21,Bowling Green,L,21.0,None,None,None,None,None,None,None,None,None,NaN
65086,2002,"Nov 9, 2002",West Virginia,W,24.0,None,Boston College,L,14.0,None,None,None,None,None,None,None,None,None,NaN
77680,2018,"Sep 15, 2018",Michigan,W,45.0,19,Southern Methodist,L,20.0,None,None,None,None,None,None,None,None,None,"Michigan Stadium - Ann Arbor, Michigan"
76593,2016,"Dec 30, 2016",None,None,None,None,None,None,None,None,Air Force,W,45.0,None,South Alabama,L,21.0,None,"Arizona Bowl (Arizona Stadium - Tucson, Arizona)"
66370,2004,"Oct 16, 2004",Arkansas State,W,13.0,None,Troy,L,9.0,None,None,None,None,None,None,None,None,None,NaN
51466,1982,"Sep 25, 1982",Northwestern,W,31.0,None,Northern Illinois,L,6.0,None,None,None,None,None,None,None,None,None,NaN
44889,1974,"Sep 21, 1974",Penn State,L,6.0,8,Navy,W,7.0,None,None,None,None,None,None,None,None,None,NaN
25992,1944,"Sep 30, 1944",Florida,L,6.0,None,Mississippi,W,26.0,None,None,None,None,None,None,None,None,None,NaN
81506,2022,"Nov 26, 2022",Alabama,W,49.0,8,Auburn,L,27.0,None,None,None,None,None,None,None,None,None,NaN
20924,1936,"Sep 26, 1936",Purdue,W,47.0,None,Ohio,L,0.0,None,None,None,None,None,None,None,None,None,NaN


In [54]:
## Save transformed data to csv in data folder
df.to_csv('../data/cfb_scrape_transformed.csv', index=False)

In [55]:
## Add conference affiliation to teams
import ast # for literal_eval function

conference_df = pd.read_csv('../data/yearly_conference_members.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})

# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


# Function to retrieve the conference based on year and team name
def get_conference(year, team_name, conference_lookup):
    """
    Given a year and team name, retrieve the conference the team belongs to.
    If no conference found, return "Independent/Unknown".
    """
    return conference_lookup.get((year, team_name), "Independent/Unknown")

# Create a lookup dictionary for team-year to conference mapping
conference_lookup = {}

for _, row in conference_df.iterrows():
    year = row['Year']
    for col, teams in row[1:].items():
        for team in teams:
            conference_lookup[(year, team)] = col

# Use funtion to assign conference affiliation the teams in each row but only if 
# the team name is not null
df['host_conference'] = df.apply(lambda row: get_conference(row['season'], row['HostTeamName'], conference_lookup) if pd.notna(row['HostTeamName']) else None, axis=1)
df['away_conference'] = df.apply(lambda row: get_conference(row['season'], row['AwayTeamName'], conference_lookup) if pd.notna(row['AwayTeamName']) else None, axis=1)
df['neutral1_conference'] = df.apply(lambda row: get_conference(row['season'], row['NeutralTeam1'], conference_lookup) if pd.notna(row['NeutralTeam1']) else None, axis=1)
df['neutral2_conference'] = df.apply(lambda row: get_conference(row['season'], row['NeutralTeam2'], conference_lookup) if pd.notna(row['NeutralTeam2']) else None, axis=1)


In [56]:
## clean up the notes column
# First Step: extract Bowl and showcase game names from the notes column
# Save special event names in the bowl_game column if it's empty
def update_bowl_game(row):
    if pd.isnull(row['bowl_game']):
        keywords = ["Championship", "Classic", "Festival", "Cup", "Bowl"]
        if pd.notnull(row['notes']) and any(keyword in row['notes'] for keyword in keywords):
            return row['notes']
    return row['bowl_game']

# create new bowl_game column
df['bowl_game'] = None
df['bowl_game'] = df.apply(update_bowl_game, axis=1)
# clear the notes column for rows that have bowl game names
df['notes'] = df.apply(lambda row: None if pd.notnull(row['bowl_game']) else row['notes'], axis=1)



In [62]:
df.sample(20)

# value counts of bowl_game column
# df['bowl_game'].value_counts()

,rk,wk,date,day,winner,winner_pts,loc_ind,loser,loser_pts,notes,...,NeutralRank2,HostTeamResult,AwayTeamResult,NeutralTeam1Result,NeutralTeam2Result,host_conference,away_conference,neutral1_conference,neutral2_conference,bowl_game
21860,286,5,"Oct 16, 1937",Sat,Texas Tech,20.0,NaN,Arizona,0.0,NaN,...,None,W,L,None,None,Border Conference,Border Conference,None,None,None
25215,354,7,"Oct 24, 1942",Sat,West Texas State,7.0,NaN,Texas-El Paso,0.0,NaN,...,None,W,L,None,None,Independent/Unknown,Independent/Unknown,None,None,None
4126,6,2,"Sep 23, 1904",Fri,Haskell,47.0,NaN,Friends,0.0,NaN,...,None,W,L,None,None,Independent/Unknown,Independent/Unknown,None,None,None
6930,272,8,"Nov 5, 1910",Sat,Colorado,44.0,@,Colorado State,0.0,NaN,...,None,L,W,None,None,Independent/Unknown,Independent/Unknown,None,None,None
76484,751,13,"Nov 19, 2016",Sat,Washington,44.0,NaN,Arizona State,18.0,"Husky Stadium - Seattle, Washington",...,None,W,L,None,None,Pac-12 Conference,Pac-12 Conference,None,None,None
28091,490,10,"Nov 15, 1947",Sat,Colorado,21.0,NaN,Wyoming,6.0,NaN,...,None,W,L,None,None,Big 8 Conference,Skyline Conference,None,None,None
20311,177,3,"Oct 5, 1935",Sat,Southern California,19.0,NaN,Pacific,7.0,NaN,...,None,W,L,None,None,Pacific Coast Conference,Independent/Unknown,None,None,None
2746,152,6,"Oct 20, 1900",Sat,Yale,38.0,NaN,Wesleyan,0.0,NaN,...,None,W,L,None,None,Independent/Unknown,Independent/Unknown,None,None,None
14346,365,7,"Oct 31, 1925",Sat,Stanford,35.0,NaN,Oregon,13.0,NaN,...,None,W,L,None,None,Pacific Coast Conference,Pacific Coast Conference,None,None,None
19197,438,8,"Nov 4, 1933",Sat,Princeton,33.0,@,Brown,0.0,NaN,...,None,L,W,None,None,Independent/Unknown,Independent/Unknown,None,None,None


In [63]:
# Save the transformed data to a csv file
df.to_csv('../data/cfb_scrape_transformed.csv', index=False)

In [58]:
# ### DATA TRANFORMATION
# # Dictionary to store unique IDs for schools
# school_id_dict = {}


# # Reorder the columns to match the desired configuration
# ordered_cols = ['HostSchoolId', 'HostScore', 'HostTeamName', 'AwaySchoolId', 'AwayScore', 
#                 'AwayTeamName', 'ContestNotes', 'HostResult', 'AwayResult', 'Wk', 
#                 'Date', 'Day', 'Winner', 'Pts', 'Loser', 'Pts.1', 'Notes', 'Year']


# # Function to generate unique IDs for school names
# def generate_school_id(name, school_id_dict):
#     if name not in school_id_dict:
#         school_id_dict[name] = len(school_id_dict) + 1
#     return school_id_dict[name]

# def transform_data_neutral(row, school_id_dict):
#     # Check if the game is a neutral site game
#     if row['Unnamed: 6'] == 'N' or (pd.notnull(row['Unnamed: 6']) and row['Unnamed: 6'] != '@'):
#         row['HostTeamName'] = row['Winner']
#         row['HostScore'] = row['Pts']
#         row['AwayTeamName'] = row['Loser']
#         row['AwayScore'] = row['Pts.1']
#         row['HostResult'] = 'Neutral'
#         row['AwayResult'] = 'Neutral'
#     # Check if the winner was the away team
#     elif row['Unnamed: 6'] == '@':
#         row['HostTeamName'] = row['Loser']
#         row['HostScore'] = row['Pts.1']
#         row['AwayTeamName'] = row['Winner']
#         row['AwayScore'] = row['Pts']
#         row['HostResult'] = 'Loss'
#         row['AwayResult'] = 'Win'
#     else:
#         row['HostTeamName'] = row['Winner']
#         row['HostScore'] = row['Pts']
#         row['AwayTeamName'] = row['Loser']
#         row['AwayScore'] = row['Pts.1']
#         row['HostResult'] = 'Win'
#         row['AwayResult'] = 'Loss'
    
#     # Generate unique IDs for schools
#     row['HostSchoolId'] = generate_school_id(row['HostTeamName'], school_id_dict)
#     row['AwaySchoolId'] = generate_school_id(row['AwayTeamName'], school_id_dict)
    
#     row['ContestNotes'] = row['Notes']
    
#     return row


In [59]:

# # Apply the transformations to each row of the dataframe with neutral site games handling
# transformed_data = data.apply(lambda row: transform_data_neutral(row, school_id_dict), axis=1)

# # Drop the 'Rk' and 'Unnamed: 6' columns
# transformed_data = transformed_data.drop(columns=['Rk', 'Unnamed: 6'])

# # Reorder the columns to match the desired configuration
# transformed_data = transformed_data[ordered_cols]
# transformed_data.head(20)



In [60]:
### Explore html structure

# ex_url = 'https://www.sports-reference.com/cfb/years/1938-schedule.html'

# ## Try using pandas to read in the table

# df = pd.read_html(ex_url)



# # covert response from list to dataframe
# df = pd.DataFrame(df[0])

# df.head()


In [61]:
# df.info()

# # Clean the dataframe of rows that are not games
# # drop rows where Wk is 'Wk'
# df = df[df['Wk'] != 'Wk']

# df.head()